# Введение в keras

In [0]:
!pip install -q keras

## Sequential Model

Обучить свою нейросеть просто! Нужно

1.   Выбрать типы и последовательность слоев
2.   Настроить параметры слоев (передав нужные в конструктор слоя или оставив вариант по умолчанию)
3.   Выбрать оптимизатор и скомпилировать модель
4.   Обучить модель!


Чтобы создать Sequential модель, просто позовите следующие строки:

In [0]:
from keras.models import Sequential

model = Sequential()

Слои теперь добавляются в модель последовательно, как будто это list:

In [0]:
from keras.layers import Dense

model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

Только что мы создали сеть с одним скрытым слоем. Сколько в ней обучаемых параметров?

Посмотреть это можно с помощью метода

In [0]:
model.summary()

*Почему столько параметров в слоях?*

*Что за `None` вместо первого элемента `Output Shape`?*

Прежде чем обучать эту модель, её нужно скомпилировать:

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

В NLP чаще всего ставятся задачи классификации, поэтому нужно запомнить такие функции потерь:


*   **categorical_crossentropy** - для многоклассовой классификации, в качестве меток должны передаваться one-hot-encoding вектора
*   **sparse_categorical_crossentropy** - аналогично предыдущему, но в качестве меток нужно передавать просто индексы соответствующих классов
*   **binary_crossentropy** - для бинарной классификации


В качестве оптимизатора обычно используют `sgd` или `adam`.


In [0]:
import numpy as np

X_train = np.random.randn(10000, 100)
y_train = np.random.randint(0, 10, size=10000)
X_test = np.random.randn(1000, 100)

In [0]:
model.fit(X_train, y_train, epochs=5, batch_size=32)

## Binary Classification

Попробуем предсказать сентимент (положительность/отрицательность) imdb'шных ревью [keras: IMDB Movie reviews sentiment classification](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)

In [0]:
from keras.datasets import imdb

In [0]:
NUM_WORDS = 10000

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

### Bag-of-words

Начнем с простейшей модели

In [0]:
def convert_to_bow(X):
  X_bow = np.zeros((len(X), NUM_WORDS))
  for i, review in enumerate(X):
    for ind in review:
      X_bow[i, ind] = 1
  return X_bow

X_train_bow, X_test_bow = convert_to_bow(X_train), convert_to_bow(X_test)

In [0]:
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_dim=NUM_WORDS))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

*Как называется такая модель?*

In [0]:
model.fit(X_train_bow, y_train, 
          batch_size=32,
          epochs=3,
          validation_data=(X_test_bow, y_test))

### Convs

Переходим к более сложным моделям.

In [0]:
from keras.preprocessing import sequence

MAX_LEN = 400

X_train_long = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test_long = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

In [0]:
from keras.layers import Embedding, Dropout, SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from keras.optimizers import Adam

EMB_DIM = 50

model = Sequential()

model.add(Embedding(input_dim=NUM_WORDS, 
                    output_dim=EMB_DIM, 
                    input_length=MAX_LEN))

model.add(Conv1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(),  # оптимизатор ещё и так можно передавать
              metrics=['accuracy'])

model.summary()

In [0]:
model.fit(X_train_long, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(X_test_long, y_test))

Попробуем улучшить качество.

Добавим предобученные словные эмбеддинги.

Чёрная магия, которую нужно для этого скастовать, выглядит как-нибудь так:

In [0]:
!pip install chakin

import chakin
chakin.search(lang='English')

chakin.download(number=11, save_dir='./')

!unzip glove.6B.zip

!pip install gensim
!python -m gensim.scripts.glove2word2vec --input glove.6B.50d.txt --output glove.6B.50d.v2w.txt

In [0]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('glove.6B.50d.v2w.txt', binary=False)

glove_word2index = {w : i for i, w in enumerate(word_vectors.index2word)}

word_index = imdb.get_word_index()
word_index_rev = {word_index[x] : x for x in word_index}

embedding_matrix = 0.05 * np.random.randn(NUM_WORDS, EMB_DIM)

num_of_known_words = 0
for word in word_index:
  ind = word_index[word] + 3
  if ind < NUM_WORDS and word in glove_word2index:
    embedding_matrix[ind] = word_vectors.vectors[glove_word2index[word]]
    num_of_known_words += 1

print('Know', num_of_known_words, 'out of', NUM_WORDS)

**Задание**: Постройте ту же самую модель, но уже с передачей весов в `Embedding`.

Обратите внимание на параметр `trainable`: во многих случаях дообучать эмбеддинги не нужно - для этого нужно передать `trainable=False`.

In [0]:
model = Sequential()

<your code. Use weights=[embedding_matrix] parameter of the Embedding layer>

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [0]:
model.fit(X_train_long, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(X_test_long, y_test))

### LSTM

Попробуем теперь использовать рекуррентную сеть.

Укоротим немного предложения для скорости:

In [0]:
MAX_LEN = 80

X_train_short = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test_short = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

**Задание**: постройте модель с LSTM.
Первым должен так и идти слой эмбеддингов, а LSTM - применяться поверх него. Результатом будет последнее предсказание LSTM - с учётом всего контекста.

Интересные параметры LSTM:
`LSTM(units=?, dropout=0.0, recurrent_dropout=0.0, return_sequences=False)`

Попробуйте `units=64` и дропауты в районе 0.2.


In [0]:
from keras.layers import LSTM, Bidirectional

<your code here>

In [0]:
model.fit(X_train_short, y_train,
          batch_size=32,
          epochs=15,
          validation_data=(X_test_short, y_test))

### LSTM-CNN

Вообще говоря, LSTM выдает не одно состояние, а много (внимание на `return_sequences`). Почему бы не попробовать использовать их все?

**Задание**: реализуйте свертки и GlobalMaxPooling поверх выхода LSTM.

In [0]:
<your code here>

## Functional API

Альтернативный (и более гибкий) вариант построения модели.

Говоря по-сложному: у каждого объекта типа Layer переопределен метод `__call__`: его можно вызывать и передавать некоторый входной тензор. Возвращаемое значение - результат применения трансформации, задаваемой этим слоем, к данному входу (опять же тензор).

А если по-простому - давайте построим ту же самую бессмысленную модель из самого начала ноутбука, но уже с новым апи.

In [0]:
from keras.models import Model
from keras.layers import Input

In [0]:
inputs = Input(shape=(100,))

hidden_layer = Dense(units=64, activation='relu')(inputs)
outputs = Dense(units=10, activation='softmax')(hidden_layer)

model = Model(inputs=inputs, outputs=outputs)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.summary()

**Задание**: сделайте модель, которая применяет свертки с шириной окна 2 и 3 к imdb dataset'у.

Пригодится слой `concatenate` для объединения результатов разных типов сверток.

In [0]:
from keras.layers import concatenate

<your code here>

## Multiclass Classification

Переходим к многоклассовой классификации: [keras: Reuters newswire topics classification](https://keras.io/datasets/#reuters-newswire-topics-classification).

У нас тут 11,228 новостей размеченных по 46 топикам.

In [0]:
from keras.datasets import reuters

NUM_WORDS = 10000

print('Loading data...')
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=NUM_WORDS,
                                                         test_split=0.2)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Mean train example len:', np.mean([len(x) for x in X_train]))
print('Mean test example len:', np.mean([len(x) for x in X_test]))

In [0]:
MAX_LEN = 150

X_train = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

**Задание**: Попробуйте обучить собственную сеть на этих данных.

Обратите внимание, что теперь уже многоклассовая классификация (вспоминаем про `sparse_categorical_crossentropy` и выходной слой с числом unit'ов, равным числу классов, и `softmax` активацией).

In [0]:
<your code here>

## Классификация на уровне символов

Попробуем вернуться к imdb и предсказывать слова, используя их символьное представление.

In [0]:
NUM_WORDS = 50000

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

word_index = imdb.get_word_index()
word_index_rev = {word_index[x] : x for x in word_index}

Зададим отображение из символов в индексы.

In [0]:
from string import punctuation

def get_range(first_symb, last_symb):
  return set(chr(c) for c in range(ord(first_symb), ord(last_symb) + 1))

chars = get_range('a', 'z') | get_range('0', '9') | set(punctuation)
char_index = {c : i for i, c in enumerate(chars)}

def get_char_index(char, char_index):
  return char_index[char] if char in char_index else len(char_index)

Используя костыли, построим тензор, в котором на месте каждого элемента стоит последовательность его символов.

In [0]:
MAX_WORD_LEN = 15
MAX_LINE_LEN = 100

X_train = sequence.pad_sequences(X_train, maxlen=MAX_LINE_LEN)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_LINE_LEN)

def build_chars_tensor(X):
  X_chars = np.zeros((len(X), MAX_LINE_LEN, MAX_WORD_LEN))
  for i, line in enumerate(X):
    for j, word_ind in enumerate(line):
      if word_ind >= 3:
        word = word_index_rev[word_ind]
        word = word if len(word) < MAX_WORD_LEN else word[-MAX_WORD_LEN:]
        X_chars[i, j, -len(word):] = [get_char_index(c, char_index) for c in word]
  return X_chars

X_chars_train = build_chars_tensor(X_train)
X_chars_test = build_chars_tensor(X_test)

*Какая размерность получится, если применить к этому тензору ещё и слой эмбеддингов?*

**Задание**: допишите код, попробуйте поклассифицировать с помощью него.

In [0]:
from keras.layers import TimeDistributed

def build_chars_layer(chars_count, char_emb_dim=20, lstm_dim=32, dropout_rate=.2):
  chars_embedding = Embedding(chars_count, char_emb_dim, name='char_embeddings')
  chars_lstm = TimeDistributed(Bidirectional(
      LSTM(lstm_dim, dropout=dropout_rate, recurrent_dropout=dropout_rate, name='char_LSTM')))
  
  def process_input(inp):
    res = chars_embedding(inp)
    return chars_lstm(res)
  return process_input
  
chars = Input(shape=(None, MAX_WORD_LEN), name='chars')

chars_level_embedding = build_chars_layer(chars_count = len(char_index) + 1)
chars_output = chars_level_embedding(chars)

<your code here>